First of all we want to clean some of the data, doing the following steps:

    - Removing the first line: It contains the extraneous text Notes offered by Prospectus
    (https://www.lendingclub.com/info/prospectus.action) instead  of the column titles, which prevents the dataset from being
    parsed by the pandas library properly
    
    - Removing the desc column: which contains a long text explanation for each loan.
    
    - Removing the url column: which contains a link to each loan on Lending Club which can only be accessed with an investor
    account
    
    - Removing all columns containing more than 50% missing values: which allows us to move faster since we can spend less time
    trying to fill these values

In [1]:
import pandas as pd
loans_2007 = pd.read_csv('LoanStats3a.csv', skiprows=1)

loans_2007 = loans_2007.drop(['desc', 'url'],axis=1)

half_count = len(loans_2007) / 2
loans_2007 = loans_2007.dropna(thresh=half_count, axis=1)

loans_2007.to_csv('loans_2007.csv', index=False)
#loans_2007.head()

C:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (0,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
loans_2007 = pd.read_csv("loans_2007.csv")
loans_2007.drop_duplicates()
print(loans_2007.iloc[0])
print(loans_2007.shape[1])

id                                1077501
member_id                      1.2966e+06
loan_amnt                            5000
funded_amnt                          5000
funded_amnt_inv                      4975
term                            36 months
int_rate                           10.65%
installment                        162.87
grade                                   B
sub_grade                              B2
emp_title                             NaN
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
issue_d                          Dec-2011
loan_status                    Fully Paid
pymnt_plan                              n
purpose                       credit_card
title                            Computer
zip_code                            860xx
addr_state                             AZ
dti                                 27.65
delinq_2yrs                       

C:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2902: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Understanding the columns (features):
We want to pay attention to any features that:
    - Leak information from the future (after the loan has already been funded)
    - Don't affect a borrower's ability to pay back a loan (e.g. a randomly generated ID value by Lending Club)
    - Formatted poorly and need to be cleaned up
    - Require more data or a lot of processing to turn into a useful feature
    - Contain redundant information
We need to especially pay attention to data leakage, since it can cause our model to overfit. This is because the model would be using data about the target column that wouldn't be available when we're using the model on future loans.

Columns to remove:
    - id: randomly generated field by Lending Club for unique identification purposes only
    - member_id: also a randomly generated field by Lending Club for unique identification purposes only
    - funded_amnt: leaks data from the future (after the loan is already started to be funded)
    - funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
    - grade: contains redundant information as the interest rate column (int_rate)
    - sub_grade: also contains redundant information as the interest rate column (int_rate)
    - emp_title: requires other data and a lot of processing to potentially be useful
    - issue_d: leaks data from the future (after the loan is already completed funded)
    - zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which
      only can be used to identify the state the borrower lives in)
    - out_prncp: leaks data from the future, (after the loan already started to be paid off)
    - out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
    - total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
    - total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
    - total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)
    - total_rec_int: leaks data from the future, (after the loan already started to be paid off),
    - total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
    - recoveries: also leaks data from the future, (after the loan already started to be paid off),
    - collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
    - last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
    - last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).

In [3]:

loans_2007 = loans_2007.drop(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"], axis=1)
loans_2007 = loans_2007.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1)
loans_2007 = loans_2007.drop(["total_rec_int", "total_rec_late_fee","pymnt_plan", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)
print("First row: ")
print(loans_2007.iloc[0])
print ("____________________________________")

print( "Number of columns: " + str(loans_2007.shape[1]))


First row: 
loan_amnt                            5000
term                            36 months
int_rate                           10.65%
installment                        162.87
emp_length                      10+ years
home_ownership                       RENT
annual_inc                          24000
verification_status              Verified
loan_status                    Fully Paid
purpose                       credit_card
title                            Computer
addr_state                             AZ
dti                                 27.65
delinq_2yrs                             0
earliest_cr_line                 Jan-1985
inq_last_6mths                          1
open_acc                                3
pub_rec                                 0
revol_bal                           13648
revol_util                          83.7%
total_acc                               9
initial_list_status                     f
last_credit_pull_d               Jul-2016
collections_12_mths_ex

Just by becoming familiar with the columns in the dataset, we were able to reduce the number of columns from 52 to 34 columns. 

We now need to decide on a target column that we want to use for modeling.
Our problem can be seen as a binary classificaiton problem with 2 possible outcomes. 0 the loan is not paid and 1 the loan is paid. To do so we can use the loan_status column, since it's the only column that directly describes if a loan was paid off on time, had delayed payments, or was defaulted on the borrower. This column contains text values and we need to convert it to a numerical one for training a model. 

In [4]:
loans_2007['loan_status'].value_counts()

Fully Paid                                             33314
Charged Off                                             5640
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  781
Does not meet the credit policy. Status:Charged Off      761
In Grace Period                                           22
Late (31-120 days)                                        17
Late (16-30 days)                                          8
Default                                                    4
Name: loan_status, dtype: int64

Description of the values: 
Fully Paid	33136	Loan has been fully paid off.
Charged Off:	5634	Loan for which there is no longer a reasonable expectation of further payments.
Does not meet the credit policy. Status:Fully Paid	1988	While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
Does not meet the credit policy. Status:Charged Off	761	While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
In Grace Period	20	The loan is past due but still in the grace period of 15 days.
Late (16-30 days)	8	Loan hasn't been paid in 16 to 30 days (late on the current payment).
Late (31-120 days)	24	Loan hasn't been paid in 31 to 120 days (late on the current payment).
Current	961	Loan is up to date on current payments.
Default	3	Loan is defaulted on and no payment has been made for more than 121 days.

From the investor's perspective, we're interested in trying to predict which loans will be paid off on time and which ones won't be. Only the Fully Paid and Charged Off values describe the final outcome of the loan. 

Since we're interesting in being able to predict which of these 2 values a loan will fall under, we can treat the problem as a binary classification one. Let's remove all the loans that don't contain either Fully Paid and Charged Off as the loan's status and then transform the Fully Paid values to 1 for the positive case and the Charged Off values to 0 for the negative case.

In [5]:
loans_2007 = loans_2007[(loans_2007['loan_status'] == "Fully Paid") | (loans_2007['loan_status'] == "Charged Off")]

status_replace = {
    "loan_status" : {
        "Fully Paid": 1,
        "Charged Off": 0,
    }
}

loans_2007 = loans_2007.replace(status_replace)

We want to remove columns that have only 1 unique value (or nan) because they don't add any value when modeling and will make our training slower

In [6]:
orig_columns = loans_2007.columns
drop_columns = []
for col in orig_columns:
    col_series = loans_2007[col].dropna().unique()
    if len(col_series) == 1:
        drop_columns.append(col)
loans_2007 = loans_2007.drop(drop_columns, axis=1)
print(drop_columns)

['initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']


Now we have to deal with columns with missing values, mathematics underlying machine learning models assumes that the data is numerical and contains no missing values.

In [7]:
null_counts=[]
null_counts = loans_2007.isnull().sum()
print(null_counts)

loan_amnt                 0
term                      0
int_rate                  0
installment               0
emp_length                0
home_ownership            0
annual_inc                0
verification_status       0
loan_status               0
purpose                   0
title                    10
addr_state                0
dti                       0
delinq_2yrs               0
earliest_cr_line          0
inq_last_6mths            0
open_acc                  0
pub_rec                   0
revol_bal                 0
revol_util               50
total_acc                 0
last_credit_pull_d        2
pub_rec_bankruptcies    697
dtype: int64


We have 3 columns with less than 50 null values and we have one column with 697 null values. There different strategies to handle null values, but for this case we will:
    - Remove entire columns with more than 1% of null values
    - Remove rows with null values

In [8]:
loans_2007 = loans_2007.drop("pub_rec_bankruptcies", axis=1)
loans = loans_2007.dropna(axis=0)
print(loans.dtypes.value_counts())

object     11
float64    10
int64       1
dtype: int64


Now we need to handle non numerical types. Machine learning algorithms can not handle non numerical types so we need to transform those values.


In [9]:
object_columns_df = loans.select_dtypes(include=["object"])
print(object_columns_df.iloc[0])
print(object_columns_df.shape)

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Jul-2016
Name: 0, dtype: object
(38892, 11)


- home_ownership: home ownership status, can only be 1 of 4 categorical values according to the data dictionary,
- verification_status: indicates if income was verified by Lending Club,
- emp_length: number of years the borrower was employed upon time of application,
- term: number of payments on the loan, either 36 or 60,
- addr_state: borrower's state of residence,
- purpose: a category provided by the borrower for the loan request,
- title: loan title provided the borrower,
- int_rate: interest rate of the loan in %,
- revol_util: revolving line utilization rate or the amount of credit the borrower is using relative to all available credit.
- earliest_cr_line: The month the borrower's earliest reported credit line was opened,
- last_credit_pull_d: The most recent month Lending Club pulled credit for this loan.

Based on the first row's values for purpose and title, it seems like these columns could reflect the same information. We should explore the unique value counts separately to confirm if this is true.

Lastly, some of the columns contain date values that would require a good amount of feature engineering for them to be potentially useful:

earliest_cr_line: The month the borrower's earliest reported credit line was opened,
last_credit_pull_d: The most recent month Lending Club pulled credit for this loan.

In [13]:
cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for col in cols:
    print(loans[col].value_counts())

RENT        18574
MORTGAGE    17216
OWN          3003
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16749
Verified           12382
Source Verified     9761
Name: verification_status, dtype: int64
10+ years    8621
< 1 year     4525
2 years      4315
3 years      4030
4 years      3367
5 years      3215
1 year       3186
6 years      2185
7 years      1720
8 years      1447
9 years      1238
n/a          1043
Name: emp_length, dtype: int64
 36 months    29041
 60 months     9851
Name: term, dtype: int64
CA    6975
NY    3727
FL    2806
TX    2680
NJ    1808
IL    1495
PA    1484
VA    1381
GA    1370
MA    1311
OH    1186
MD    1031
AZ     854
WA     827
CO     774
NC     759
CT     733
MI     713
MO     674
MN     605
NV     483
SC     464
WI     444
OR     439
AL     438
LA     431
KY     316
OK     292
KS     261
UT     255
AR     239
DC     210
RI     197
NM     187
WV     173
NH     168
HI     167
DE     113
MT      83
WY      82
AK      

The home_ownership, verification_status, emp_length, term, and addr_state columns all contain multiple discrete values. We should clean the emp_length column and treat it as a numerical one since the values have ordering (2 years of employment is less than 8 years).

Based on the first row's values for purpose and title, it seems like these columns could reflect the same information. Let's look at the unique value counts for the purpose and title columns to understand which column we want to keep.

In [15]:
print(loans["purpose"].value_counts())
print("__________________")
print(loans["title"].value_counts())

debt_consolidation    18216
credit_card            5054
other                  3879
home_improvement       2916
major_purchase         2164
small_business         1779
car                    1519
wedding                 935
medical                 682
moving                  578
vacation                376
house                   371
educational             320
renewable_energy        103
Name: purpose, dtype: int64
__________________
Debt Consolidation                         2119
Debt Consolidation Loan                    1648
Personal Loan                               644
Consolidation                               497
debt consolidation                          490
Credit Card Consolidation                   354
Home Improvement                            347
Debt consolidation                          324
Small Business Loan                         312
Credit Card Loan                            309
Personal                                    302
Consolidation Loan               

The home_ownership, verification_status, emp_length, and term columns each contain a few discrete categorical values. We will encode these columns as dummy variables and keep them.

It seems like the purpose and title columns do contain overlapping information but we'll keep the purpose column since it contains a few discrete values. In addition, the title column has data quality issues since many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation).

Lastly, the addr_state column contains many discrete values and we'd need to add 49 dummy variable columns to use it for classification. This would make our Dataframe much larger and could slow down how quickly the code runs. Let's remove this column from consideration.



In [16]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

loans = loans.drop(["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)
loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")
loans = loans.replace(mapping_dict)

We erred on the side of being conservative with the 10+ years, < 1 year and n/a mappings. We assume that people who may have been working more than 10 years have only really worked for 10 years. We also assume that people who've worked less than a year or if the information is not available that they've worked for 0. This is a general heuristic but it's not perfect.

Let's now encode the home_ownership, verification_status, title, and term columns as dummy variables so we can use them in our model.

In [17]:
cat_columns = ["home_ownership", "verification_status", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns])
loans= pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)

In [18]:
print(loans.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38892 entries, 0 to 39785
Data columns (total 38 columns):
loan_amnt                              38892 non-null float64
int_rate                               38892 non-null float64
installment                            38892 non-null float64
emp_length                             38892 non-null int64
annual_inc                             38892 non-null float64
loan_status                            38892 non-null int64
dti                                    38892 non-null float64
delinq_2yrs                            38892 non-null float64
inq_last_6mths                         38892 non-null float64
open_acc                               38892 non-null float64
pub_rec                                38892 non-null float64
revol_bal                              38892 non-null float64
revol_util                             38892 non-null float64
total_acc                              38892 non-null float64
home_ownership_MORTGAGE    

So our objective here is to answer the question: Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not? 
The approach we will choose here is we will model this problem as a binary classification problem, where 0 means the borrower doesn't pay back on time and 1 meaning the borrower does pay on time.

Now we need to find an error metric, an error metric will tell us how our alogirthm is performing and how we can improve it.
Our objective in this is to make money -- we want to fund enough loans that are paid off on time to offset our losses from loans that aren't paid off. An error metric will help us determine if our algorithm will make us money or lose us money.

In this case, we're primarily concerned with false positives and false negatives. Both of these are different types of misclassifications. With a false positive, we predict that a loan will be paid off on time, but it actually isn't. This costs us money, since we fund loans that lose us money. With a false negative, we predict that a loan won't be paid off on time, but it actually would be paid off on time. This loses us potential money, since we didn't fund a loan that actually would have been paid off.

Since we're viewing this problem from the standpoint of a conservative investor, we need to treat false positives differently than false negatives. A conservative investor would want to minimize risk, and avoid false positives as much as possible. It's better to miss an opportunity (false negatives) than funding a risky loan (false positives).

let's say we've correctly identified loan_status in 85.7% of cases. However, we've done this by predicting 1 for every row. What this means is that we'll actually lose money. Let's say we loan out 1000 dollars on average to each borrower. Each borrower pays us 10% interest back. So we make a projected profit of 100 dollars on each loan. But we lose 1000 on one. Therefore we lose money.

That means that in this case, we don't want to use accuracy (number of correct predictions), and should instead use metrics that tell us the number of false positives and false negatives.

This means that we should optimize for:

high recall (true positive rate) what percentage of loans that could be funded would I fund?
low fall-out (false positive rate) what percentage of the loans that I fund would not be repaid?


Into machine learning! The first algorithm we will try is logistic regression: 
- it's quick to train and we can iterate more quickly,
- it's less prone to overfitting than more complex models like decision trees,
- it's easy to interpret.

In [23]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
cols = loans.columns
train_cols = cols.drop("loan_status")
features = loans[train_cols]
target = loans["loan_status"]
lr.fit(features, target)
predictions = lr.predict(features)


In [34]:
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

fpr = fp/(fp+tn)

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

tpr = tp/(float)(tp+fn)

print(tpr)

0.998647402398


Cross validation splits the dataset into groups, then makes predictions on each group using the other groups as training data. This ensures that we don't overfit by generating predictions on the same data that we train our algorithm with.

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import cross_val_predict, KFold
lr = LogisticRegression()
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (float)(tp + fn)
fpr = fp / (float)(fp + tn)

print(tpr)
print(fpr)

32486
5466
44
7
0.998647402398
0.99872099397


It looks like our classifier works really well but if the model was predicting all ones we would get the same result. This is because of the imbalance of the classes. There are 6 times as many loans that were paid off on time (1), than loans that weren't paid off on time (0). This causes a major issue when we use accuracy as a metric. This is because due to the class imbalance, a classifier can predict 1 for every row, and still have high accuracy. Also our model will tend to predict more towards 1 than what it should really be.

The two main ways two approeach class imablance are:

- Use oversampling and undersampling to ensure that the classifier gets input that has a balanced number of each class.
- Tell the classifier to penalize misclassifications of the less prevalent class more than the other class. t

The first method tries to force the rows so we have the same number of classes either by throwing out many rows of data.
Copy rows multiple times or generate fake data. In our case neither of this is a good idea.

Instead we are going to do the second approach the penalty means that the logistic regression classifier pays more attention to correctly classifying rows where loan_status is 0. This lowers accuracy when loan_status is 1, but raises accuracy when loan_status is 0.

And it's very easy to do with scikit

In [36]:
lr = LogisticRegression(class_weight="balanced")
kf = KFold(features.shape[0], random_state=1)

predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (float)(tp + fn)
fpr = fp / (float)(fp + tn)

print(tpr)
print(fpr)

0.629757147249
0.63073268774


Let's try a more complex algorithm. Random forests.


In [41]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_predict
rf = RandomForestClassifier(class_weight="balanced", random_state=1)
kf = KFold(features.shape[0], random_state=1)
predictions = cross_val_predict(rf, features, target, cv=kf)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (float)(tp + fn)
fpr = fp / (float)(fp + tn)

print(tpr)
print(fpr)

0.970273593606
0.966563128083
